# 資料前處理(Label encoding、 One hot encoding)
這兩個編碼方式的目的是為了將類別 (categorical)或是文字(text)的資料轉換成數字，而讓程式能夠更好的去理解及運算。
> Label encoding : 把每個類別 mapping 到某個整數，不會增加新欄位

> One hot encoding : 為每個類別新增一個欄位，用 0/1 表示是否

![](images/Encoder.PNG)


## Encoding Categorical features (or label)
![](images/Encoding.PNG)


In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]})
df

,blood,Y,Z
0,A,high,NaN
1,B,low,NaN
2,AB,high,-1196.0
3,O,mid,72.0
4,B,mid,83.0


# 方法一：sklearn - label encoder + onehot encoder
>onehot encoder要用2D array，若維度所以要用reshape(-1,1)<br>
>onehot encoder要數字，若資料文文字要先用label encoder轉數字

In [3]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder() # 之後可以直接以encoder使用
encoded_Y = encoder.fit_transform(df['blood']) # 放入要轉換的欄位
print(encoded_Y) # 有4種血型 變成 0 1 2 3

df['blood'] = encoded_Y # 轉換完的再放回原表格df, 資料轉換成有次序的資料
df

[0 2 1 3 2]


,blood,Y,Z
0,0,high,NaN
1,2,low,NaN
2,1,high,-1196.0
3,3,mid,72.0
4,2,mid,83.0


In [4]:
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder() # 需要給他numpy的格式。官網：list of array
#print(type(df['blood'])) # try看看上一個是什麼格式(pandas)
d = np.array(df['blood'])
d

array([0, 2, 1, 3, 2])

In [5]:
d.ndim # 看是幾維：1 維

onehot_df = onehot.fit_transform(d.reshape(-1, 1)) # ❓fit_tranform需要 2D 維度 so 要.reshape
#onehot_df # 結果不常見==> 看一下type
print(onehot_df)
print('-'*50)
print(onehot_df.ndim)
#type(onehot_df) # type = scipy => 還是要改成array 
onehot_df = onehot.fit_transform(d.reshape(-1, 1)).toarray() # 改成array
print('-'*15,'one hot encoding', '-'*15)
print(onehot_df)

  (0, 0)	1.0
  (1, 2)	1.0
  (2, 1)	1.0
  (3, 3)	1.0
  (4, 2)	1.0
--------------------------------------------------
2
--------------- one hot encoding ---------------
[[1. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]]


## One hot encoding
One Hot encoding的編碼邏輯為將類別拆成多個行(column)，每個列中的數值由1、0替代，當某一列的資料存在的該行的類別則顯示1，反則顯示0。

然在指定column進行編碼的情形下，One hot encoding<b>無法直接對字串進行編碼，必須先透過Label encoding將字串以數字取代後再進行One hot encoding處理。</b>

> categorical_features = [0]: 表示欲在data上執行One hot encoding的index為0

> data_le: 為經過Label encoding編碼的資料(註:OneHotEncoder的輸入要為2-D array，而Label encoding為1-D array)


OneHotEncoder會轉出scipy.csr_matrix資料結構用.toarray()轉array
從結果可以知道：

數字0的column 代表的是A

數字1的column 代表的是B

數字2的column 代表的是AB

🌟除了轉換字串外，One hot encoding也可以轉換數字。在此處的data就不需要先經過Label encoding編碼

```python
# importing one hot encoder from sklearn 
# There are changes in OneHotEncoder class 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 
   
# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])], 
                                      remainder='passthrough') 
  
data = np.array(columnTransformer.fit_transform(data), dtype = str) 
```

In [6]:
# importing one hot encoder from sklearn 
# There are changes in OneHotEncoder class 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer 

# creating one hot encoder object with categorical feature 0 
# indicating the first column 
columnTransformer = ColumnTransformer([('encoder', 
                                        OneHotEncoder(), 
                                        [0])],
                                     remainder='passthrough') 
# [0]針對第0個欄位
# remainder='passthrough其他欄位會直接被忽略 可以試試拿掉後的差異
 
data = np.array(columnTransformer.fit_transform(df), dtype = str) # 也可以用另一方式轉成array：結尾.toarray()
# toarray出現錯誤是原本data就已經是array了
data #❓

array([['1.0', '0.0', '0.0', '0.0', 'high', 'nan'],
       ['0.0', '0.0', '1.0', '0.0', 'low', 'nan'],
       ['0.0', '1.0', '0.0', '0.0', 'high', '-1196.0'],
       ['0.0', '0.0', '0.0', '1.0', 'mid', '72.0'],
       ['0.0', '0.0', '1.0', '0.0', 'mid', '83.0']], dtype='<U7')

In [7]:
data_pd = pd.DataFrame(data) # 變回一開始的pandas
data_pd

,0,1,2,3,4,5
0,1.0,0.0,0.0,0.0,high,nan
1,0.0,0.0,1.0,0.0,low,nan
2,0.0,1.0,0.0,0.0,high,-1196.0
3,0.0,0.0,0.0,1.0,mid,72.0
4,0.0,0.0,1.0,0.0,mid,83.0


# 方法二：Keras - label encoder + to_categorical
>to_categorical要數字，若資料文文字要先用label encoder轉數字

In [10]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils  # ❌ No module named 'keras' ❌

df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]});

# label encoder 
encoder = LabelEncoder()
encoded_Y = encoder.fit_transform(df['blood']) # 放入要轉換的欄位
print(encoded_Y) # 有4種血型 變成 0 1 2 3
df['blood'] = encoded_Y # 轉換完的再放回原表格df, 資料轉換成有次序的資料
df

# convert integers to one hot encoding
keras_onehot = np_utils.to_categorical(encoded_Y)
keras_onehot

ModuleNotFoundError: No module named 'keras'

## 方法三：pd.get_dummies方法
![](images/Encoding_pd.PNG)
pd.get_dummies(df)
>get_dummies可以直接轉字串，反而無法轉換數字<br>
>get_dummies沒指定columns，會全部轉換

In [13]:
# pandas 內建
df = pd.DataFrame({'blood':['A','B','AB','O','B'], 
                   'Y':['high','low','high','mid','mid'],
                   'Z':[np.nan,np.nan,-1196,72,83]})

df1 = pd.get_dummies(df)
df1

,Z,blood_A,blood_AB,blood_B,blood_O,Y_high,Y_low,Y_mid
0,NaN,1,0,0,0,1,0,0
1,NaN,0,0,1,0,0,1,0
2,-1196.0,0,1,0,0,1,0,0
3,72.0,0,0,0,1,0,0,1
4,83.0,0,0,1,0,0,0,1


In [14]:
# 指定欄位:
df2 = pd.get_dummies(df.blood)
df2

,A,AB,B,O
0,1,0,0,0
1,0,0,1,0
2,0,1,0,0
3,0,0,0,1
4,0,0,1,0


## 練習一：sklearn - label encoder + onehot encoder
下面的資料可以看到country那欄皆為字串， 大部分的模型都是基於數學運算，字串無法套入數學模型進行運算，<br>
在此先對其進行Label encoding編碼，我們從 sklearn library中導入 LabelEncoder class，對第一行資料進行fit及transform並取代之。

In [15]:
import numpy as np
import pandas as pd
country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

,Country,Age,Salary
0,Taiwan,25,20000
1,Australia,30,32000
2,Ireland,45,59000
3,Australia,35,60000
4,Ireland,22,43000
5,Taiwan,36,52000


## 練習二：Keras - label encoder + to_categorical

In [16]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

ModuleNotFoundError: No module named 'keras'

## 練習三：Pandas.get_dummies
>　get_dummies : 僅能將字串轉換為One hot encoding表示形式， 沒指定columns會全部轉換。

In [17]:
country=['Taiwan','Australia','Ireland','Australia','Ireland','Taiwan']
age=[25,30,45,35,22,36]
salary=[20000,32000,59000,60000,43000,52000]
dic={'Country':country,'Age':age,'Salary':salary}
data=pd.DataFrame(dic)
data

,Country,Age,Salary
0,Taiwan,25,20000
1,Australia,30,32000
2,Ireland,45,59000
3,Australia,35,60000
4,Ireland,22,43000
5,Taiwan,36,52000
